In [1]:
from gurobipy import *
from prettytable import PrettyTable
from math import pi
import numpy as np

In [2]:
OPF = Model('3 bus OPF')

In [3]:
#Sets
G = 3 # Generators
B = 3 # Bus
L = 3 #Lines
T = 1
Gen = range(G)
Bus = range(B)
Line = range(L)
Period = range(T)

### Input the susceptance for each transmission line

In [4]:
B1 = {(0,1):{'B':-10 ,'Limit':1000 },
      (0,2):{'B':-8 ,'Limit':1000 },
      (1,2):{'B':-5 ,'Limit':1500 } }

### Generate the graph

In [5]:
y = list(B1.keys())
x=list(map (lambda t: (t[1], t[0]), y))
B2=dict(zip(x,B1.values()))
B3 = {**B1, **B2}

In [6]:
tuple_list = list(B3.keys())
desired_list = [tuple(list(tup)+[t])  for t in Period for tup in tuple_list]
desired_list
vals=[B3[(i,j)] for (i,j) in B3.keys()]*T
B = dict(zip(desired_list,vals))
B

{(0, 1, 0): {'B': -10, 'Limit': 1000},
 (0, 2, 0): {'B': -8, 'Limit': 1000},
 (1, 2, 0): {'B': -5, 'Limit': 1500},
 (1, 0, 0): {'B': -10, 'Limit': 1000},
 (2, 0, 0): {'B': -8, 'Limit': 1000},
 (2, 1, 0): {'B': -5, 'Limit': 1500}}

In [7]:
L = {(0,0):200, (1,0):550, (2,0):100, (0,1):200, (1,1):550, (2,1):100} # electircal Load (MW)
Sbase = 100 # Power Base (MVA)

In [8]:
GenAt = {Gen[0]:{'a':0.001562 ,'b':7.92 ,'c':561, 'Pmin':150 ,'Pmax':600},
         Gen[1]:{'a':0.001940 ,'b':7.85 ,'c':310, 'Pmin':100 ,'Pmax':400},
         Gen[2]:{'a':0.004820 ,'b':7.97 ,'c':78,  'Pmin':50 , 'Pmax':200}}
GenAt

{0: {'a': 0.001562, 'b': 7.92, 'c': 561, 'Pmin': 150, 'Pmax': 600},
 1: {'a': 0.00194, 'b': 7.85, 'c': 310, 'Pmin': 100, 'Pmax': 400},
 2: {'a': 0.00482, 'b': 7.97, 'c': 78, 'Pmin': 50, 'Pmax': 200}}

In [9]:
Pmin = [(GenAt[i]['Pmin']) for i in Gen]
Pmax = [(GenAt[i]['Pmax']) for i in Gen]
g = [(a,t)  for t in Period for a in Gen]
b = [(a,t)  for t in Period for a in Bus] 

In [10]:
P = OPF.addVars(g, name = 'Pgen')
U = OPF.addVars(g, vtype = GRB.BINARY, name = 'Status')
Delta = OPF.addVars(b, lb = -6*T, ub = 6*T, name = 'Delta')
Pij = OPF.addVars(B, lb = [-B[(i,j,t)]['Limit']  for (i,j,t) in B.keys()], 
                  ub = [B[(i,j,t)]['Limit'] for (i,j,t) in B.keys()], name = 'PF')
U

{(0, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 0): <gurobi.Var *Awaiting Model Update*>,
 (2, 0): <gurobi.Var *Awaiting Model Update*>}

In [11]:
PowerFlow = OPF.addConstrs((Pij[(i,j,t)] == Sbase*(Delta[i,t]-Delta[j,t])*(B[(i,j,t)]['B']) for (i,j,t) in Pij.keys()), name = 'PowerFlow')
NetBus = OPF.addConstrs( (P[(i,t)] - (L[i,t])  == Pij.sum(i,'*',t) for (i,t) in g ), name = 'NetBus')

PMAX = OPF.addConstrs((P[(i,t)] <= GenAt[i]['Pmax']*U[(i,t)] for (i,t) in g), name = 'Pmax')
PMIN = OPF.addConstrs((P[(i,t)] >= GenAt[i]['Pmin']*U[(i,t)] for (i,t) in g), name = 'Pin')

for t in Period:
    Delta[(0,t)].lb = 0
    Delta[(0,t)].ub = 0

In [12]:
OF = quicksum(GenAt[i]['c']+GenAt[i]['b']*P[(i,t)]+GenAt[i]['a']*P[(i,t)]*P[(i,t)] for (i,t) in g)

In [13]:
OPF.setObjective(OF, GRB.MINIMIZE)
OPF.Params.Method=-1
OPF.setParam( 'OutputFlag', True )
OPF.optimize()



status = OPF.status
if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE \
or status == GRB.Status.UNBOUNDED:
    OPF.computeIIS()
    OPF.write("model.ilp")
OPF.write("model.lp")

Parameter Method unchanged
   Value: -1  Min: -1  Max: 5  Default: -1
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 15 rows, 15 columns and 39 nonzeros
Model has 3 quadratic objective terms
Variable types: 12 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [8e+00, 8e+00]
  QObjective range [3e-03, 1e-02]
  Bounds range     [1e+00, 2e+03]
  RHS range        [1e+02, 6e+02]
Presolve removed 11 rows and 10 columns
Presolve time: 0.00s
Presolved: 4 rows, 5 columns, 10 nonzeros
Presolved model has 3 quadratic objective terms
Variable types: 4 continuous, 1 integer (1 binary)

Root relaxation: objective 8.194356e+03, 10 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 8194.35612    0    1          - 8194.35612      -     -    0s
H    0     0      

In [14]:
OPF.printStats()


Statistics for model 3 bus OPF :
  Linear constraint matrix    : 15 Constrs, 15 Vars, 39 NZs
  Variable types              : 12 Continuous, 3 Integer (3 Binary)
  Matrix coefficient range    : [ 1, 1000 ]
  Objective coefficient range : [ 7.85, 7.97 ]
  Variable bound range        : [ 1, 1500 ]
  RHS coefficient range       : [ 100, 550 ]


In [15]:
OPF.getVars()

[<gurobi.Var Pgen[0,0] (value 393.16983694560304)>,
 <gurobi.Var Pgen[1,0] (value 334.603755313934)>,
 <gurobi.Var Pgen[2,0] (value 122.22640774046309)>,
 <gurobi.Var Status[0,0] (value 1.0)>,
 <gurobi.Var Status[1,0] (value 1.0)>,
 <gurobi.Var Status[2,0] (value 1.0)>,
 <gurobi.Var Delta[0,0] (value 0.0)>,
 <gurobi.Var Delta[1,0] (value 0.1581775966009732)>,
 <gurobi.Var Delta[2,0] (value 0.043740300430787665)>,
 <gurobi.Var PF[0,1,0] (value 158.1775966009732)>,
 <gurobi.Var PF[0,2,0] (value 34.99224034463013)>,
 <gurobi.Var PF[1,2,0] (value -57.21864808509277)>,
 <gurobi.Var PF[1,0,0] (value -158.1775966009732)>,
 <gurobi.Var PF[2,0,0] (value -34.99224034463013)>,
 <gurobi.Var PF[2,1,0] (value 57.21864808509277)>]

In [16]:
tuple_list = [(0,1), (0,2), (1,2)]
time = range(5)

In [17]:
desired_list = [tuple(list(tup)+[t])  for t in time for tup in tuple_list]

In [18]:
desired_list

[(0, 1, 0),
 (0, 2, 0),
 (1, 2, 0),
 (0, 1, 1),
 (0, 2, 1),
 (1, 2, 1),
 (0, 1, 2),
 (0, 2, 2),
 (1, 2, 2),
 (0, 1, 3),
 (0, 2, 3),
 (1, 2, 3),
 (0, 1, 4),
 (0, 2, 4),
 (1, 2, 4)]